## Running multiple ML models at the same time
This notebook is an example how to run multiple models side-by-side and combine results of all models. A video stream from a local camera is processed by the hand, face, and person detection models. Combined result is then displayed.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

**Access to camera is required to run this sample.**

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `video_source` or in [env.ini](../../env.ini) file by defining `VIDEO_SOURCE` variable and assigning `video_source = None`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify camera index 

In [ ]:
video_source = None        # camera index or URL; 0 to use default local camera, None to take from env.ini file

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, degirum_tools

degirum_tools.configure_colab() # configure for Google Colab

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = degirum_tools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, degirum_tools.get_cloud_zoo_url(), degirum_tools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
hand_det_model = zoo.load_model("yolo_v5s_hand_det--512x512_quant_n2x_orca1_1")
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca1_1")
person_det_model = zoo.load_model("yolo_v5s_person_det--512x512_quant_n2x_orca1_1", overlay_line_width = 1)

# AI prediction loop, press 'x' or 'q' to stop video
with degirum_tools.Display("Hands and Faces") as display:
    for res in degirum_tools.predict_stream(
        degirum_tools.CombiningCompoundModel(degirum_tools.CombiningCompoundModel(hand_det_model, face_det_model), person_det_model), 
        video_source
    ):
        display.show(res.image_overlay)
